<div style="background-color:#D2DCFA; padding:10px;">
    
# Описание задачи.
    
    
Интернет-магазин **«Викишоп»** запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
    
---

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества **F1** не меньше **0.75**. 
    
---
    
    
# Описание данных
    
Признаки:
    
 - **text** - комментарии
    
 - **toxic** - целевая переменная (0 - обычный комментарий, 1 - токсичный)
    
    
---
    
    
# Моделирование
    
Будем использовать несколько моделей:
    
 - LogisticRegression в качестве бейзлайна
    
 - RandomForestClassifier
    
 - LightGBM
    
    
В качестве метрики выступит **`F1`**

---

## Шаг 1. Загрузка данных

Импортируем необходимые библиотеки

In [1]:
!pip install spacy

In [2]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/12.8 MB 3.6 MB/s eta 0:00:04
     ----- ---------------------------------- 1.8/12.8 MB 3.9 MB/s eta 0:00:03
     --------- ------------------------------ 3.1/12.8 MB 4.5 MB/s eta 0:00:03
     --------------- ------------------------ 5.0/12.8 MB 5.7 MB/s eta 0:00:02
     -------------------- ------------------- 6.6/12.8 MB 6.0 MB/s eta 0:00:02
     ----------------------- ---------------- 7.6/12.8 MB 5.9 MB/s eta 0:00:01
     -------------------------- ------------- 8.4/12.8 MB 5.5 MB/s eta 0:00:01
     ---------------------------- ----------- 9.2/12.8 MB 5.3 MB/s eta 0:00:01
     --------------------------------- ------ 10.7/12.8 MB 5.5 MB/s eta 0:00:01
     ---------------------------------------  12.6/12.8 MB 6.0 MB/s eta 0:00:01
     ---------------------------------------- 12.8/12.8 MB 5.9 M

In [3]:
import pandas as pd 
import numpy as np 
import re
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import spacy

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline


from tqdm.notebook import tqdm 


tqdm.pandas()

In [4]:
start_time = time.time()

Загрузим данные

In [5]:
try:
    df = pd.read_csv(r"C:\Users\ilyal\OneDrive\Рабочий стол\Портфолио\practicum_projects\vikishop_NLP_12\toxic_comments.csv")
    
except:
    df = pd.read_csv(r"/datasets/toxic_comments.csv")

In [6]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Выведем информацию о датафрейме

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Проверим датафрейм на наличие явных дубликатов

In [8]:
df.duplicated().sum()

0

Сразу посмотрим присутствует ли дисбаланс классов

In [9]:
df.toxic.value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

<div style="background-color:#D2DCFA; padding:10px;">
    
### Промежуточный вывод.
    
 - были загружены данные
    
 - пропусков и дубликатов в датафрейме нет
    
 - присутствует явный дисбаланс классов
    
 - в текстах присутствуют переносы строк, ссылки, email-адреса, различные символы, аббревиатуры и цифры

---

## Шаг 2. Обработка данных

Создадим функцию, которая с помощью регулярных выражений будет убирать из текста переносы строк, ссылки, адреса электронной почты, различные символы, аббревиатуры и цифры

In [10]:
def clean_text(text):
    text = text.replace('\n', ' ')  # удаление новых строк
    # удаление ссылок
    text = re.sub(r'http\S+', '', text) 
    text = re.sub(r'www\.\S+', '', text)
    text = re.sub(r'\S+@\S+\.\S+', '', text) # удаление email адресов
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # оставляем только латинские буквы
    text = text.lower()  # приведение к нижнему регистру
    text = text.replace('utc', ' ') # удаление аббревиатуры
    return text

Применим функцию к тексту

In [11]:
df['cleaned_text'] = df['text'].apply(clean_text)

In [12]:
df.sample(5)

,Unnamed: 0,text,toxic,cleaned_text
47885,47940,"""\nThe point is that it's already been covered...",0,the point is that it s already been covered ...
55849,55910,"OK, the first human I have found. I want to co...",0,ok the first human i have found i want to co...
29644,29683,Looking through the cites the Qualudes id refe...,0,looking through the cites the qualudes id refe...
105682,105779,The MOS covers a lot of material and im not qu...,0,the mos covers a lot of material and im not qu...
99744,99841,No need to kick them where they are down. Your...,0,no need to kick them where they are down your...


Преобразование прошло успешно

Теперь проведем токенизацию, лемматизацию (приведение слов к их изначальной форме), а так же удалим стоп-слова

In [13]:
# Загрузка модели English для SpaCy
nlp = spacy.load("en_core_web_sm")

def lem_tok_text(text):
    # Приводим текст к нижнему регистру
    text = text.lower()
    
    # Обработка текста с помощью SpaCy
    doc = nlp(text)
    
    # Удаление стоп-слов и лемматизация
    lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    # Возвращаем строку из лемматизированных слов
    return ' '.join(lemmatized_tokens)

Применим функцию к "чистому тексту"

In [14]:
df['processed_text'] = df['cleaned_text'].progress_apply(lem_tok_text)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [15]:
df.head()

,Unnamed: 0,text,toxic,cleaned_text,processed_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation edit username hardcore metallica f...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he matches this background colour i m s...,d aww match background colour m seemingly st...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not trying to edit war it...,hey man m try edit war s guy constantly re...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestions on im...,t real suggestion improvement wonder sec...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember...,sir hero chance remember page s


Преобразование прошло успешно

Проведем повторную проверку на наличие явных дубликатов после лемматизации

In [16]:
df['processed_text'].duplicated().sum()

726

Удалим дубликаты

In [17]:
df = df.drop_duplicates(subset='processed_text')

In [18]:
df['processed_text'].duplicated().sum()

0

Так же необходимо убрать из текста все сокращенные названия месяцев

In [19]:
months_short = r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)\b'

def remove_months(text):
    return re.sub(months_short, '', text)

# применим функцию
df['processed_text'] = df['processed_text'].apply(remove_months)

Теперь проведем векторизацию текстов, для этого будем использовать TfidfVectorizer

Для этого сразу разделим данные на обучающую и тестовую выборки

In [20]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'],
                                                    df['toxic'],
                                                    test_size=0.1, 
                                                    random_state=42)

In [21]:
tfidf = TfidfVectorizer(max_features=5000)

X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

print(f'Размер обучающей выборки {X_train.shape[0]} строк и {X_train.shape[1]} признаков')
print(f'Размер тестовой выборки {X_test.shape[0]} строк и {X_test.shape[1]} признаков')

Размер обучающей выборки 142709 строк и 5000 признаков
Размер тестовой выборки 15857 строк и 5000 признаков


In [22]:
type(X_train)

scipy.sparse._csr.csr_matrix

Преобразование прошло успешно

<div style="background-color:#D2DCFA; padding:10px;">
    
## Промежуточный вывод
    
 - были созданы функции для очистки текста с помощью регулярных выражений
    
 - была проведена лемматизация текста
    
 - были удалены явные дубликаты, появившиеся после лемматизации
    
 - было проведено разделение данных на тренировочную и тестовую выборки
    
 - была проведена векторизация текста с помощью TfidfVectorizer

---

## Шаг 3.Обучение моделей

Проверим, влияет ли балансировка классов на метрику. Для балансировки будем использовать SMOTE

### Обучение с балансировкой

Создадим пайплайн imblearn, в качестве бейзлайна будем использовать Логистичесукю Регрессию

In [23]:
pipeline = Pipeline([
    ('smote', SMOTE()),
    ('model_lr', LogisticRegression(max_iter=1000))
])

param_grid = {
    'model_lr__C': [0.001, 0.01, 0.1, 1, 10],
    'model_lr__solver': ['liblinear', 'lbfgs']
}

Инициализируем SKFold и разобьем данные на 5 слоев со стратификацией

In [24]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

Скопируем метрику F1 для использования в поиске по сетке

In [25]:
f1_scorer = make_scorer(f1_score)

Инициализируем поиск по сетке

In [26]:
gs_bal = GridSearchCV(pipeline,
                  param_grid,
                  scoring=f1_scorer,
                  cv=skf,
                  verbose=1)

Обучим поиск по сетке

In [27]:
%%time 
gs_bal.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
CPU times: total: 4min 47s
Wall time: 3min 16s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('smote', SMOTE()),
                                       ('model_lr',
                                        LogisticRegression(max_iter=1000))]),
             param_grid={'model_lr__C': [0.001, 0.01, 0.1, 1, 10],
                         'model_lr__solver': ['liblinear', 'lbfgs']},
             scoring=make_scorer(f1_score, response_method='predict'),
             verbose=1)

In [28]:
gs_bal.best_score_

0.6656547042758059

Как видно, метрика с балансировкой сильно ниже пороговой

---

### Обучение без балансировки

Теперь запустим поиск по сетке без балансировки

In [29]:
pipeline = Pipeline([
    ('model_lr', LogisticRegression(max_iter=1000))
])

param_grid = {
    'model_lr__C': [0.001, 0.01, 0.1, 1, 10],
    'model_lr__solver': ['liblinear', 'lbfgs']
}

In [30]:
gs = GridSearchCV(pipeline,
                  param_grid,
                  scoring=f1_scorer,
                  cv=skf,
                  verbose=1)

In [31]:
%%time
gs.fit(X_train, y_train) 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
CPU times: total: 54.4 s
Wall time: 18.1 s


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('model_lr',
                                        LogisticRegression(max_iter=1000))]),
             param_grid={'model_lr__C': [0.001, 0.01, 0.1, 1, 10],
                         'model_lr__solver': ['liblinear', 'lbfgs']},
             scoring=make_scorer(f1_score, response_method='predict'),
             verbose=1)

In [32]:
gs.best_score_

0.773868432705538

Так стало намного лучше

### В дальнейшем не будем балансировать классы

---

Теперь обучим модели и подберем гиперпараметры

In [33]:
pipeline_final = Pipeline([
    ('model', LogisticRegression(max_iter=1000))
])

Список моделей и их гиперпараметров для перебора

In [34]:
param_grid_final = [
    {
        'model': [LogisticRegression(max_iter=1000)],
        'model__C': [0.01, 0.1, 1, 5, 10],
        'model__solver': ['liblinear', 'lbfgs']
    },
    {
        'model': [RandomForestClassifier(random_state=42)],
        'model__n_estimators': [200, 300],
        'model__max_depth': [None, 10],
        'model__min_samples_split': [2, 5, 10]
    },
    {
        'model': [lgb.LGBMClassifier(random_state=42)],
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.02],
        'model__max_depth': [-1, 10, 20]
    }
]

Создание стратифицированных фолдов. Будем делать 3 разбиения, чтобы сократить время обучения моделей

In [35]:
cv = StratifiedKFold(n_splits=3,
                     shuffle=True,
                     random_state=42)

Инициализация поиска по сетке

In [36]:
gs_final = GridSearchCV(pipeline_final,
                        param_grid_final,
                        cv=cv,
                        scoring=f1_scorer,
                        n_jobs=-1)

Подбор лучших параметров

In [37]:
%%time 
gs_final.fit(X_train, y_train)

C:\Users\ilyal\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


CPU times: total: 4.69 s
Wall time: 50min 8s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('model',
                                        LogisticRegression(max_iter=1000))]),
             n_jobs=-1,
             param_grid=[{'model': [LogisticRegression(max_iter=1000)],
                          'model__C': [0.01, 0.1, 1, 5, 10],
                          'model__solver': ['liblinear', 'lbfgs']},
                         {'model': [RandomForestClassifier(random_state=42)],
                          'model__max_depth': [None, 10],
                          'model__min_samples_split': [2, 5, 10],
                          'model__n_estimators': [200, 300]},
                         {'model': [LGBMClassifier(random_state=42)],
                          'model__learning_rate': [0.01, 0.1, 0.02],
                          'model__max_depth': [-1, 10, 20],
                          'model__n_estimators': [50, 100, 200]}],
             scoring=make_scorer(f1_score, response_method='predict'))

Выведем лучшую метрику с кросс-валидации

In [38]:
gs_final.best_score_

0.7732309767827078

Выведем лучшую модель

In [39]:
gs_final.best_params_

{'model': LogisticRegression(max_iter=1000),
 'model__C': 10,
 'model__solver': 'lbfgs'}

Сохраним результаты кросс-валидации в датафрейм

In [40]:
res = pd.DataFrame(gs_final.cv_results_)

Выведем топ-10 моделей и их метрики

In [41]:
res[['param_model', 'rank_test_score', 'mean_test_score']].sort_values(by='rank_test_score').reset_index(drop=True).head(10)

,param_model,rank_test_score,mean_test_score
0,LogisticRegression(max_iter=1000),1,0.773231
1,LogisticRegression(max_iter=1000),2,0.769795
2,LogisticRegression(max_iter=1000),3,0.769357
3,LogisticRegression(max_iter=1000),4,0.767881
4,LGBMClassifier(random_state=42),5,0.762453
5,RandomForestClassifier(random_state=42),6,0.762127
6,RandomForestClassifier(random_state=42),7,0.761860
7,RandomForestClassifier(random_state=42),8,0.761375
8,RandomForestClassifier(random_state=42),9,0.761297
9,RandomForestClassifier(random_state=42),10,0.761069


<div style="background-color:#D2DCFA; padding:10px;">
    
## Промежуточный вывод
    
 - было проверено влияние дисбаланса классов на модель
    
 - был сделан вывод о том, что в балансировке классов нет необходимости, достаточно стратификации при обучении
    
 - были обучены 3 модели на кросс-валидации, лучшую метрику показала Логистическая регрессия **`0.773`**

---

## Шаг 4. Предсказание

Инициализируем лучшую модель и сделаем предсказание на тестовой выборке

In [42]:
model = LogisticRegression(solver='lbfgs', C=10, max_iter=1000)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f'F1 score for LogisticRegression: {f1_score(y_test, y_pred)}')

F1 score for LogisticRegression: 0.7761398697291738


<div style="background-color:#D2DCFA; padding:10px;">
    
## Промежуточный вывод
    
Было выполнено предсказание на тестовых данных с помощью LogisticRegression, метрика составила **`0.776`**

---

<div style="background-color:#D2DCFA; padding:10px;">
    
# Финальный вывод
    
---
    
### Загрузка данных
    
 - были загружены данные
    
 - пропусков и дубликатов в датафрейме нет
    
 - присутствует явный дисбаланс классов
    
 - в текстах присутствуют переносы строк, ссылки, email-адреса, различные символы, аббревиатуры и цифры
   
---
    
### Предобработка данных
    
 - были созданы функции для очистки текста с помощью регулярных выражений
    
 - была проведена лемматизация текста
    
 - были удалены явные дубликаты, появившиеся после лемматизации
    
 - было проведено разделение данных на тренировочную и тестовую выборки
    
 - была проведена векторизация текста с помощью TfidfVectorizer
    
---
    
### Обучение моделей
    
 - было проверено влияние дисбаланса классов на модель
    
 - был сделан вывод о том, что в балансировке классов нет необходимости, достаточно стратификации при обучении
    
 - были обучены 3 модели на кросс-валидации, лучшую метрику показала Логистическая регрессия **`0.773`**
    
---
    
### Предсказание на тестовых данных
    
Было выполнено предсказание на тестовых данных с помощью LogisticRegression, метрика составила **`0.776`**
    
---
    
### Реккомендации

Исходя из результатов исследования я бы рекомендовал к выбору именно Логистическую регрессию. Её главное преимущество по сравнению с другими моделями  в том, что она тратит в десятки раз меньше времени на обучение. При этом в данном случае, с предоставленными комментариями, с данными параметрами моделей, Логистическая регрессия показывает метрику F1 выше всех.

In [43]:
end_time = time.time()
print(end_time - start_time)

5064.382776737213
